# Labels and Labeling

In this project, **labels** are one of the most important pieces of the dataset.

Why? Because a label is the part of the data that describes the **future outcome**.

Until now, everything we created (IB levels, AVWAPs, slopes, states, etc.) was based only on:

- what happened **in the past**, or
- what is known **at the current minute**

That is exactly what we want for features:
> **Features must be built without looking into the future.**

But this is a **strategy testing** project.  
So we also need a way to measure:

- “What happened AFTER this minute?”
- “Was this condition followed by an up move or a down move?”
- “How large was the move?”

That is what **labeling** gives us.


## Why labels are necessary

For any hypothesis test or strategy evaluation, we need two things:

1. **Inputs (features / states):** information available up to time *t*  
2. **Outputs (labels):** what happens after time *t* (the future)

Without labels, we cannot objectively test whether our signals have predictive value.

So in this step, we create columns that explicitly represent the future, such as:

- return after 15 minutes
- return after 30 minutes
- direction after 15 minutes
- direction after 30 minutes


## The label columns we create (and why)

### **1) `close_f15`, `close_f30` — future close prices**

These columns store the **future close price** relative to the current minute:

- **`close_f15`** = the close price **15 minutes later**
- **`close_f30`** = the close price **30 minutes later**

**Why do we need them?**

Because these values are the clean reference points we use to compute:

- future returns (`ret15`, `ret30`)
- future direction labels (`dir15`, `dir30`)

So you can think of them as the **raw future targets**.


### **2) `ret15`, `ret30` — future return percentages**

These columns measure **how much the price changes** from now until the future point:

- **`ret15`** = percent return from now → 15 minutes later  
- **`ret30`** = percent return from now → 30 minutes later  

A return tells us **magnitude**, not just direction.

**Why do we need returns?**

Because many hypotheses are not only about “up or down”, but also about:

- How **strong** the move is
- Whether a pattern leads to a **meaningful** change, not a tiny fluctuation

So returns are critical for:

- statistical comparisons (mean/median return by condition)
- evaluating signal strength
- defining thresholds (e.g., “move is significant if return > X”)


### **3) `dir15`, `dir30` — directional labels (up/down)**

These are simple classification-style labels:

- **`dir15`** = direction after 15 minutes  
- **`dir30`** = direction after 30 minutes  

Usually encoded as:

- **1** = price went **up**
- **0** = price went **down or did not increase**  
  (exact rule depends on how we define “up” vs “down”, but the idea is binary direction)

**Why do we need directional labels?**

Because many strategy questions are naturally phrased as:

- “Does this setup predict an **up move** more often than random?”
- “When state X happens, is the next 15 minutes more likely to be **up** or **down**?”

Directional labels allow:

- hit-rate / accuracy style analysis
- hypothesis tests on probabilities (e.g., win-rate > 50%)
- later ML classification (in the next stage of the project)


## Key principle: labels must not leak into features

A crucial rule:

- **Features must use only past/current information**
- **Labels can use future information**
- But labels must be created carefully so that future data does not “leak” back into feature calculations

So labeling is the step where we **intentionally** add future-looking columns, but we keep them clearly separated in purpose:

- **Features / states** → describe the present and past  
- **Labels** → describe the future outcome we are trying to predict


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Now I can use our cleaned data, all features and states with AVWAP levels and IB levels which we made in "03_features_states.ipynb"

PROJECT_ROOT = Path("..").resolve()

DATA_CACHE = PROJECT_ROOT / "data" / "cache"

CACHE_FILE = DATA_CACHE / "spy_1min_et_clean_with_completed_all_features_states.csv"

df_lb = pd.read_csv(CACHE_FILE, parse_dates=['datetime'])

df_lb.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_px_up_last5,slope_down_sign,cross_px_down,cross_px_down_last5,cross_av_ud,cross_av_ud_last5,cross_av_ou,cross_av_ou_last5,cross_av_od,cross_av_od_last5
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,0,0,0,0,0,0,0,0,0,0
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,0,0,0,0,0,0,0,0,0,0
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,0,0,0,0,0,0,0,0,0,0
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,0,0,0,0,0,0,0,0,0,0
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,0,0,0,0,0,0,0,0,0,0


In [3]:
H15, H30 = 15, 30

g = df_lb.groupby(df_lb["datetime"].dt.normalize(), sort=False) # we are making sure our future returns just give information about intraday
# without this operation, our end of day candles might show future returns in the next day.
# This is intraday trading strategy testing project and we DON'T WANT that. 

df_lb["close_f15"] = g["close"].shift(-H15) # close price after 15 1-min candles just in that trading day
df_lb["close_f30"] = g["close"].shift(-H30) # close price after 15 1-min candles just in that trading day

df_lb["ret15"] = (df_lb["close_f15"] / df_lb["close"]) - 1.0  # percentage return after 15 1-min candles
df_lb["ret30"] = (df_lb["close_f30"] / df_lb["close"]) - 1.0  # percentage return after 30 1-min candles
# -1 represents -(current price / current price) format which is the second term of (future price/current price) - (current price/current price)

df_lb["dir15"] = (df_lb["ret15"] > 0).astype(int) # if return is positive this column returns 1, return is negative this column returns 0
df_lb["dir30"] = (df_lb["ret30"] > 0).astype(int) # if return is positive this column returns 1, return is negative this column returns 0

df_lb.head(16)


,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_ou,cross_av_ou_last5,cross_av_od,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,0,0,0,0,648.42,648.24,0.000247,-0.000031,1,0
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,0,0,0,0,648.28,647.97,0.000015,-0.000463,1,0
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,0,0,0,0,648.11,648.27,-0.000231,0.000015,0,1
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,0,0,0,0,648.57,648.24,0.000262,-0.000247,1,0
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.000580,NaN,NaN,649.06,...,0,0,0,0,648.66,648.29,-0.000008,-0.000578,0,0
5,2025-09-08 09:35:00,648.88,648.62,648.780,38252,0.000401,0.000469,NaN,NaN,649.06,...,0,0,0,0,648.27,648.28,-0.000786,-0.000771,0,0
6,2025-09-08 09:36:00,648.92,648.78,648.790,36436,0.000216,0.000419,NaN,NaN,649.06,...,0,0,0,0,648.50,648.48,-0.000447,-0.000478,0,0
7,2025-09-08 09:37:00,649.06,648.80,648.870,35151,0.000401,0.000388,NaN,NaN,649.06,...,0,0,0,0,648.51,648.57,-0.000555,-0.000462,0,0
8,2025-09-08 09:38:00,648.91,648.23,648.230,52975,0.001049,0.000524,NaN,NaN,649.06,...,0,0,0,0,648.37,648.62,0.000216,0.000602,1,1
9,2025-09-08 09:39:00,648.35,648.06,648.110,58512,0.000447,0.000503,NaN,NaN,649.06,...,0,0,0,0,648.76,648.42,0.001003,0.000478,1,1


## 4) Saving the labeled dataframe as a `.csv` file in `data/cache/`

At this point, we have finished the **labeling step**.

That means our dataframe now contains:

- All previously created feature columns (IB levels, AVWAPs, slopes, states, etc.)
- Plus the **future-looking label columns**, such as:
  - **`close_f15`, `close_f30`**
  - **`ret15`, `ret30`**
  - **`dir15`, `dir30`**

Now we want to **save this updated version** as a new `.csv` file.


### Why do we save it in `data/cache/`?

We store it in:

- **`data/cache/`**

because this file is an **intermediate “ready-to-use” dataset** that later notebooks can load directly.

Saving it here helps us:

- Avoid recalculating labels every time
- Keep a clean pipeline where each notebook produces a clear output file
- Use the labeled dataset repeatedly for:
  - hypothesis testing
  - statistical summaries
  - later evaluation work


### Why is this notebook its own `.ipynb` file?

This labeling step is not just “another small edit”.

It is a **major research boundary** in the project, because:

- Before labeling, we build features using only **past/current information**
- With labeling, we introduce **future outcomes** on purpose, as targets for testing

This is directly connected to the core statistics idea used in hypothesis testing:

- **Measured statistics** (example: average future return after a certain setup)
- **Sampling distributions** (what results look like across many repeated samples)
- **p-values** (how likely our observed result is under a null/random scenario)

In other words, once labels exist, we can finally test questions like:

- “Is the win-rate above 50%?”
- “Is the average return significantly different from 0?”
- “Is this pattern statistically meaningful, or could it be random noise?”

So this notebook deserves to be a separate `.ipynb` because it creates the dataset that is **directly used for statistical hypothesis testing**.



In [4]:
from pathlib import Path

# 1) Define project root which is the main branch in our repository
PROJECT_ROOT = Path("..").resolve()

# 2) We need to go to data/cache folder so define that pathway
DATA_CACHE = PROJECT_ROOT / "data" / "cache"
DATA_CACHE.mkdir(parents=True, exist_ok=True)

clean_csv_path = DATA_CACHE / "spy_1min_et_clean_with_labeled.csv"

df_lb.to_csv(clean_csv_path, index=False)

print("Saved CSV to:", clean_csv_path)

Saved CSV to: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/cache/spy_1min_et_clean_with_labeled.csv
